# Rice Crop disease identifier
This projects itends to assist farmers with identifying rice crop disease using Machine Learning model

## Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

import sklearn
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import skimage
from skimage.io import imread,imshow
from skimage.feature import hog
from skimage.transform import rescale,resize

## Importing the dataset

In [ ]:
src_path={'Bacterial Blight Disease':'Rice_Diseases/Bacterial Blight Disease',
'Blast Disease':'Rice_Diseases/Blast Disease',
'Brown Spot Disease':'Rice_Diseases/Brown Spot Disease',
'False Smut Disease':'Rice_Diseases/False Smut Disease',
'Tungro':'Rice_Diseases/Tungro'}

bacterial_blight=[]
blast=[]
brown_spot=[]
false_smut=[]
tungro=[]

for disease, path in src_path.items():
    for file in os.listdir(path):
        if disease=='Bacterial Blight Disease':
            bacterial_blight.append(os.path.join(path, file))
        if disease=='Blast Disease':
            blast.append(os.path.join(path, file))
        if disease=='Brown Spot Disease':
            brown_spot.append(os.path.join(path, file))
        if disease=='False Smut Disease':
            false_smut.append(os.path.join(path, file))
        if disease=='Tungro':
            tungro.append(os.path.join(path, file))

In [ ]:
print(bacterial_blight)
print(blast)
print(brown_spot)
print(false_smut)
print(tungro)

In [ ]:
print(len(bacterial_blight))
print(len(blast))
print(len(brown_spot))
print(len(false_smut))
print(len(tungro))

## Visualisation

In [ ]:
bacterial_sample_image=bacterial_blight[0]
blast_sample_image=blast[0]
brown_spot_sample_image=brown_spot[0]
false_smut_sample_image=false_smut[0]

fig=plt.figure(figsize=(15,8))
fig.add_subplot(1,4,1)
# plt.axis('off')
plt.title('Bacterial Blight Disease Sample Image')
imshow(bacterial_sample_image)

fig.add_subplot(1,4,2)
# plt.axis('off')
plt.title('Blast Disease Sample Image')
imshow(blast_sample_image)

fig.add_subplot(1,4,3)
# plt.axis('off')
plt.title('Bacterial Blight Disease Sample Image')
imshow(brown_spot_sample_image)

fig.add_subplot(1,4,4)
# plt.axis('off')
plt.title('Bacterial Blight Disease Sample Image')
imshow(false_smut_sample_image)

## Preprocess

In [ ]:
combined_path_dataset=[]
for path in bacterial_blight:
    combined_path_dataset.append(path)

for path in blast:
    combined_path_dataset.append(path)

for path in brown_spot:
    combined_path_dataset.append(path)

for path in false_smut:
    combined_path_dataset.append(path)

for path in tungro:
    combined_path_dataset.append(path)

In [ ]:
path=combined_path_dataset[12]
image=imread(path,as_gray=True)
rescaled_image=resize(image,(200,200))
# dasify,daisified_image=daisy(rescaled_image,step=180, radius=58, rings=2, histograms=6, orientations=8, visualize=True)
# fig=plt.figure(figsize=(10,6))
# plt.subplot(1,2,1)
# imshow(rescaled_image)
# plt.subplot(1,2,2)
# imshow(image)
# print(dasify.shape)
hogify, hog_img = hog(
    image, orientations=8,
    pixels_per_cell=(4, 4),
    cells_per_block=(8,8),
    visualize=True
)

fig, ax = plt.subplots()
ax.axis("off")
ax.imshow(hog_img)
ax.set_title(f"Hog Image")
plt.show()
print(hogify)
print(hogify.shape)

In [ ]:
print(len(combined_path_dataset))

In [ ]:
hogified_images=[]
for path in combined_path_dataset:
    image=imread(path,as_gray=True)
    resized_image=resize(image,(200,200))
    hogify= hog(resized_image, 
                orientations=8,
                pixels_per_cell=(8, 8),
                cells_per_block=(16,16)
                )
    print(hogify.shape)
    hogified_images.append(hogify)


## Building Dataset and Training Model 

In [ ]:
x=hogified_images
y_bb=[]
y_bd=[]
y_bs=[]
y_fs=[]
y_t=[]
y=[]
for i in range(0,len(bacterial_blight)):
    y_bb.append(1)

for i in range(0,len(blast)):
    y_bd.append(2)

for i in range(0,len(brown_spot)):
    y_fs.append(3)

for i in range(0,len(false_smut)):
    y_bs.append(4)

for i in range(0,len(tungro)):
    y_t.append(5)

In [ ]:
y=y_bb+y_bd+y_fs+y_bs+y_t
print(x[0].shape)
print(x[1])
print(y)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Ensure you do not overwrite built-in names
data_length = int(len(x) * 0.8)
start_index = 0

for i in range(0, data_length, 100):
    xval = x[i:i+100]
    yval = y[i:i+100]
    
    mlpc = MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', hidden_layer_sizes=(150, 150, 150), 
                         learning_rate='adaptive', max_iter=100, solver='adam', tol=1e-8, verbose=True, warm_start=True)

    mlpc.fit(xval, yval)


In [ ]:
ypred=mlpc.predict(xtest)
print(f"accuracy:{accuracy_score(ypred,ypred)}")

## Deploying the model

In [ ]:
# ['bz2', 'gzip', 'lzma', None, 'pickle', 'zipfile']
joblib.dump(mlpc,open("model_100.pkl",'wb'))


In [ ]:
mlpc=joblib.load(open('model_100.pkl','rb'))